In [10]:
import pandas as pd
import re
import operator

df = pd.read_csv(
    "./court_1_requiredOCR_False.csv", usecols=["판례내용", "file_name"]
)
pd.set_option("display.max_rows", 8000)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 20000)
print(df.shape)

with open("./crime_extract.txt", "r") as f:
    crime_ext = f.read()
crime_regex = re.compile(
    r"(\b사 *건\b)?(?(1)|(\b\d+[가-힣]{1,3}\d+\b))(?P<target>.*?)(\n *피 *고 *인\b)?(?(4)|(\b피 *고 *인\b))",
    re.DOTALL,
)

print('--')
def extract_crime_1(x):
    try:
        crime = crime_regex.search(x["판례내용"]).group("target")
        # crime = re.split(r'\n사 *건\b|\b사 *건 *번 *호\b|\n피 *고 *인\b', maxsplit=2, string=x['판례내용'])[1]
        # crime = re.search(r'(\b사 *건\b)?(?(1)|(\b\d+[가-힣]{1,3}\d+\b))(?P<target>.*?)(\n *피 *고 *인\b)?(?(4)|(\b피 *고 *인\b))', x['판례내용'], re.DOTALL).group('target')

    except AttributeError:
        return "x"
        # return x['판례내용']
    return crime


def extract_crime_2(x):
    text = x["사건-피고인"]
    text = re.sub(r"\n", "", text)
    crime = re.findall(rf"{crime_ext}", text)
    crime = list(set(crime))
    return crime


df["사건-피고인"] = df.apply(extract_crime_1, axis=1)
print('--')
df["test"] = df.apply(extract_crime_2, axis=1)
print(df.shape)
# df = df[list(map(operator.not_,[True if i != 'x' else False for i in df['사건-피고인']]))]
# df = df[[True if i != 'x' else False for i in df['사건-피고인']]]
df = df[[True if len(i) != 0 else False for i in df["test"]]]
print(df.shape)
# print(df[['사건-피고인',"file_name"]])
print(df[["test", "사건-피고인"]])
df.to_csv(
    "../csv_arrangement/test3.csv", index=False, columns=["test","사건-피고인"]
)


(6024, 2)
--
--
(6024, 4)
(3545, 4)
                                                   test                                             사건-피고인
1                                                [교통사고]                            2011고정1311  교통사고처리특례법위반
2                                                  [사기]                             2019고단363   사기, 변호사법위반
3                                                  [횡령]                                     2007고단5780  횡령
4                                                  [상해]                                     2020고단1142  상해
5                                                  [사기]                                      2008고정2440 사기
6                                                  [상해]                            2016고단3997   상해, 공무집행방해
7     [라. 도로교통법위반(무면허운전), 나. 도로교통법위반(사고후미조치), 가. 교통사...   2017고단387  가. 교통사고처리특례법위반(치상)\n             나...
8                                                [강도상해]                                    2014고합256  강도상해
9